In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [ ]:
f1 = 90
f2 = 110
fs = 500
n = np.arange(0, 100)
n_unif = n[::1]
N = 10000

tensor_data = np.zeros((N,  len(n_unif),2 , 2))

for s_idx in range(N):
    rand_idx = np.random.permutation(len(n))[:len(n_unif)]
    n_rand = n[rand_idx]
    t_rand = n_rand / fs
    # t_rand = (np.random.randn(len(n_rand))/3 +  n_rand)/fs
    a_rand = np.cos(2 * np.pi * f1 * t_rand)
    tensor_data[s_idx, :, 0, 0] = t_rand
    tensor_data[s_idx, :, 1, 0] = a_rand
    
    rand_idx = np.random.permutation(len(n))[:len(n_unif)]
    n_rand = n[rand_idx]
    t_rand = n_rand / fs
    # t_rand = (np.random.randn(len(n_rand))/3 +  n_rand)/fs
    a_rand = np.cos(2 * np.pi * f2 * t_rand)
    tensor_data[s_idx, :, 0, 1] = t_rand
    tensor_data[s_idx, :, 1, 1] = a_rand

# data_matrix = tensor_data.reshape((2,2 * len(n_unif), N))

data_matrix = np.concatenate((tensor_data[...,0], tensor_data[...,1]), axis=0)
labels = np.hstack((np.zeros(N), np.ones(N)))

In [ ]:
batch_size = 64
# Convert data and labels to PyTorch tensors
data = torch.tensor(data_matrix, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.long).squeeze()  # Assuming labels are in the shape (N, 1)
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
# Split the data into training and testing sets (80% train, 20% test)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader_graph = DataLoader([Data(x=x[:,1:2], pos=x[:,0:1],y=torch.tensor([y])) for x,y in zip(X_train, y_train)], batch_size=batch_size, shuffle=True)
test_loader_graph = DataLoader([Data(x=x[:,1:2], pos=x[:,0:1],y=torch.tensor([y])) for x,y in zip(X_test, y_test)], batch_size=batch_size)

In [ ]:
X_train[0]

In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import MLP, EdgeConv, DynamicEdgeConv, global_max_pool




class GNN(torch.nn.Module):
    def __init__(self, hidden_size, output_size,k):
        super().__init__()
        self.k = k
        self.aggr = 'max'
        self.conv1 = DynamicEdgeConv(MLP([2 * 2,hidden_size,hidden_size]), self.k, self.aggr)
        self.conv2 = DynamicEdgeConv(MLP([2 * hidden_size,hidden_size,hidden_size]), self.k, self.aggr)
        # self.conv2 = EdgeConv(MLP([2 * 64, 128]), aggr)
        self.lin1 = Linear(hidden_size + hidden_size, 1024)

        self.mlp = MLP([ 1024, 512, 256, output_size], dropout=0.5,
                       batch_norm=False)

    def forward(self, data):
        pos = data.pos
        p = data.x
        x0 = torch.cat([pos,p],dim=1)
        if(data.batch is not None):
            batch = data.batch
            x1 = self.conv1(x0, batch)
            x2 = self.conv2(x1, batch)
            out1 = self.lin1(torch.cat([x1, x2], dim=1))
            out2 = global_max_pool(out1, batch)
        else:       
            x1 = self.conv1(x0)
            x2 = self.conv2(x1)
            out1 = self.lin1(torch.cat([x1, x2], dim=1))
            out2 = out1.sum(dim=-2, keepdim=out1.dim() == 2)
        out = self.mlp(out2)
        # inters = {'x1': x1, 'x2': x2, 'out1': out1,'out2': out2, 'out': out}
        return F.log_softmax(out, dim=1) #, inters



# Initialize the model, loss function, and optimizer
input_size = np.prod(data.shape[1:])
hidden_size = 64
output_size = len(torch.unique(labels))
model = GNN(hidden_size, output_size,k=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
num_epochs = 100
for epoch in (range(num_epochs)):
    model.train()
    total_correct = 0
    total_samples = 0
    for batch_x in train_loader_graph:
        batch_x = batch_x.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_x.y)
        loss.backward()
        optimizer.step()

        # Calculate training accuracy for this batch
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == batch_x.y).sum().item()
        total_samples += batch_x.y.size(0)

    # Calculate and print training accuracy for the epoch
    train_accuracy = (total_correct / total_samples) * 100
    tqdm.write(f'Epoch [{epoch + 1}/{num_epochs}], Train Accuracy: {train_accuracy:.2f}%')

# Evaluation on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x in test_loader_graph:
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        _, predicted = torch.max(outputs, 1)
        total += batch_x.y.size(0)
        correct += (predicted == batch_x.y).sum().item()

# Calculate test accuracy
test_accuracy = (correct / total) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:

# Create DataLoader for training and testing data
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.fc15 = nn.Linear(hidden_size, hidden_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc15(x)
        x = self.relu(x)
        x = self.fc2(x)
        return torch.softmax(x, dim=1)

# Initialize the model, loss function, and optimizer

input_size = np.prod(data.shape[1:])
hidden_size = 64
output_size = len(torch.unique(labels))
model = MLP(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 400
for epoch in (range(num_epochs)):
    model.train()
    total_correct = 0
    total_samples = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x.view(-1,input_size))
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        # Calculate training accuracy for this batch
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == batch_y).sum().item()
        total_samples += batch_y.size(0)

    # Calculate and print training accuracy for the epoch
    train_accuracy = (total_correct / total_samples) * 100
    tqdm.write(f'Epoch [{epoch + 1}/{num_epochs}], Train Accuracy: {train_accuracy:.2f}%')

# Evaluation on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        outputs = model(batch_x.view(-1,input_size))
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

# Calculate test accuracy
test_accuracy = (correct / total) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')